<img src="https://github.com/rjpost20/Onramp-Project/blob/main/data/pexels-vishnu-r-nair-1105666.jpg?raw=true">
Image by <a href="https://www.pexels.com/@vishnurnair/" >Vishnu R Nair</a> on <a href="https://www.pexels.com/photo/people-at-concert-1105666/" >Pexels.com</a>

# *Onramp x Vanguard Spotify Project*

## Notebook 2: Data Transformation and Storage

## *By Ryan Posternak*

<br>

## Table of Contents:

* Step 1: Data Ingestion
 * Establish Connection to Spotify's API
 * Part 1: Artists Dataframe
 * Part 2: Albums Dataframe
 * Part 3: Tracks Dataframe
 * Part 4: Track Features Dataframe
<br><br>
* __Step 2: Data Transformation__
 * __Part 1: Handling Null/Missing Values__
 * __Part 2: Deduplication__
   * __2.1: Remove duplicate albums__
   * __2.2: Remove duplicate songs__
<br><br>
* __Step 3: Storage__
 * __Part 1: artist Table__
 * __Part 2: album Table__
 * __Part 3: track Table__
 * __Part 4: track_feature Table__
<br><br>
* Step 4: Analytics / Visualizations
 * Part 1: SQL Query Analytics
   * 1.1: Top songs by artist in terms of `duration_ms`
   * 1.2: Top artists in the database by number of `followers`
   * 1.3: Top songs by artist in terms of `tempo`
   * 1.4: Average `track_feature` values by artist
   * 1.5: Max and min average `track_feature` and `duration_ms` values by album
   * 1.6: Proportion of `explicit` tracks by artist
   * 1.7: Median `track_feature` values by artist
 * Part 2: Data Visualizations
   * 2.1: Mean `track_feature` values by `genre`
   * 2.2: Distribution of `track_feature` and `duration_ms` values
   * 2.3: Evolution of artists, visualized
   * 2.4: Evolution of genres, visualized
   * 2.5: Correlations between `track_feature` + `duration_ms` + `followers` values

## Imports

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import os, collections

import sqlite3

<br>

# Step 2: Data Transformation

### Import pandas `pkl` files

In [2]:
artists_df = pd.read_pickle('data/artists_df_unprocessed.pkl')

albums_df = pd.read_pickle('data/albums_df_unprocessed.pkl')

tracks_df = pd.read_pickle('data/tracks_df_unprocessed.pkl')

track_features_df = pd.read_pickle('data/track_features_df_unprocessed.pkl')

<br>

## Part 1: Handling Null / Missing Values

Below I create function to print a count of null values per column that we can use for each dataframe, as well as to print a Pandas-style "info" table

In [3]:
def display_df_null_info(dataframe):
    null_columns = [column for column in dataframe.columns if dataframe[column].isna().sum() > 0]
    
    print('Features with missing values:')
    
    if not artists_df[null_columns].isna().sum().shape[0]:  # Print 'NONE' if no columns with missing values
        print('NONE', '\n')
    else:
        print(artists_df[null_columns].isna().sum(), '\n')  # Print columns names with missing values, if any
    print('Blank values:', \
    
    # Print total number of blank values in dataframe
    sum([dataframe.loc[dataframe[column] == ''].shape[0] for column in dataframe.columns]), '\n\n')
    
    return dataframe.info()

In [4]:
display_df_null_info(artists_df)

Features with missing values:
NONE 

Blank values: 0 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   artist_id     20 non-null     object
 1   artist_name   20 non-null     object
 2   external_url  20 non-null     object
 3   genre         20 non-null     object
 4   image_url     20 non-null     object
 5   followers     20 non-null     int64 
 6   popularity    20 non-null     int64 
 7   type          20 non-null     object
 8   artist_uri    20 non-null     object
dtypes: int64(2), object(7)
memory usage: 1.5+ KB


**Remarks:**
- Looks good - no null/blank values and table schema matches the provided format.

In [5]:
display_df_null_info(albums_df)

Features with missing values:
NONE 

Blank values: 0 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   album_id      150 non-null    object
 1   album_name    150 non-null    object
 2   external_url  150 non-null    object
 3   image_url     150 non-null    object
 4   release_date  150 non-null    object
 5   total_tracks  150 non-null    int64 
 6   type          150 non-null    object
 7   album_uri     150 non-null    object
 8   artist_id     150 non-null    object
dtypes: int64(1), object(8)
memory usage: 10.7+ KB


**Remarks:**
- No null values, but we'll need to correct the datatype of the `release_date` feature, from object to datetime.

In [6]:
albums_df.release_date = albums_df.release_date.astype('datetime64')
assert albums_df.release_date.dtype == 'datetime64[ns]'

In [7]:
display_df_null_info(tracks_df)

Features with missing values:
NONE 

Blank values: 0 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2152 entries, 0 to 2151
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   track_id      2152 non-null   object
 1   song_name     2152 non-null   object
 2   external_url  2152 non-null   object
 3   duration_ms   2152 non-null   int64 
 4   explicit      2152 non-null   bool  
 5   disc_number   2152 non-null   int64 
 6   type          2152 non-null   object
 7   song_uri      2152 non-null   object
 8   album_id      2152 non-null   object
dtypes: bool(1), int64(2), object(6)
memory usage: 136.7+ KB


**Remarks:**
- Everything looks good, no null values and Pandas correctly picked up on the `bool` datatype of the `explicit` feature.

In [8]:
display_df_null_info(track_features_df)

Features with missing values:
NONE 

Blank values: 0 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2152 entries, 0 to 2151
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   track_id          2152 non-null   object 
 1   danceability      2152 non-null   float64
 2   energy            2152 non-null   float64
 3   instrumentalness  2152 non-null   float64
 4   liveness          2152 non-null   float64
 5   loudness          2152 non-null   float64
 6   speechiness       2152 non-null   float64
 7   tempo             2152 non-null   float64
 8   type              2152 non-null   object 
 9   valence           2152 non-null   float64
 10  song_uri          2152 non-null   object 
dtypes: float64(8), object(3)
memory usage: 185.1+ KB


**Remarks:**
- Last table is good to go - no null or blank values and datatypes all match the provided format. That was easy!

<br>

## Part 2: Deduplication

### 2.1: Remove duplicate albums

Below I print each artist and a list of their respective albums in `albums_df`. I then manually check each one to look for potential duplicate albums. Removing these duplicate albums should substantially reduce the number of duplicate songs that will need to be checked in the next part of the deduplication process.

While this is a relatively labor intensive process, it would be difficult to find a more automated way to do this. To show the difficulty of automating the task, take for example Ellie Goulding's 4th and 5th albums, Halcyon and Halcyon Days. Who's to say whether these are duplicate albums or not? Looking it up shows that they are: Halcyon Days is a repackage of the original album with new songs added. But that would be impossible to know simply by looking for matching words. Additionally, because we're only dealing with 20 artists here, manual review will not take that long.

I do not do in-depth research of every album to look for potential duplicate songs - just a quick check for anything obvious.

In [9]:
for artist_id in albums_df.artist_id.unique():
    # Display each artist name above list of albums
    print(artists_df.loc[artists_df['artist_id'] == artist_id].artist_name)
    # Display artist's list of albums in albums_df
    print(list(albums_df.loc[albums_df['artist_id'] == artist_id].album_name.values))
    print('\n')

0    Beyoncé
Name: artist_name, dtype: object
['RENAISSANCE', 'The Lion King: The Gift [Deluxe Edition]', 'Lemonade', 'BEYONCÉ [Platinum Edition]', '4', 'I Am...World Tour', 'Above And Beyoncé Dance Mixes', 'I AM...SASHA FIERCE', "B'Day Deluxe Edition", "B'Day"]


1    Billie Eilish
Name: artist_name, dtype: object
['Happier Than Ever', 'WHEN WE ALL FALL ASLEEP, WHERE DO WE GO?']


2    Bob Dylan
Name: artist_name, dtype: object
['Oh Mercy', 'Down In The Groove', 'Knocked Out Loaded', 'Empire Burlesque', 'Infidels', 'Shot Of Love', 'Saved', 'Slow Train Coming', 'Street-Legal', 'Desire']


3    Bob Marley & The Wailers
Name: artist_name, dtype: object
['Confrontation', 'Uprising', 'Survival', 'Kaya (40th Anniversary Edition)', 'Kaya - Deluxe Edition', 'Babylon By Bus', 'Exodus', 'Rastaman Vibration', 'Natty Dread', 'Catch A Fire']


4    Cuco
Name: artist_name, dtype: object
['Fantasy Gateway', 'Para Mi', 'Songs4u', 'wannabewithu']


5    Doja Cat
Name: artist_name, dtype: object
['Plan

**Duplicate album findings:**

0. Beyonce
- 'I Am...World Tour' (Live-recorded versions of songs)
- 'Above And Beyoncé Dance Mixes' (Remix album of songs already in dataframe)
- "B'Day" (Deluxe edition already in dataframe)
<br><br>
1. Billie Eilish
- No duplicates
<br><br>
2. Bob Dylan
- No duplicates
<br><br>
3. Bob Marley
- 'Kaya - Deluxe Edition' (album already in dataframe)
<br><br>
4. Cuco
- No duplicates
<br><br>
5. Doja Cat
- No duplicates
<br><br>
6. Drake
- No duplicates
<br><br>
7. Ellie Goulding
- 'Lights 10' (Repeat of songs in Lights)
- 'Halcyon' (supplanted by Halcyon Days)
- 'Bright Lights (Lights Re-pack / Bonus Version)' (Remixed/bonus version)
<br><br>
8. J. Cole
- "Revenge Of The Dreamers III: Director's Cut" (Director's Cut version of album already in dataframe)
<br><br>
9. Jack Johnson
- 'En Concert' (Live-recorded album)
- 'Sleep Through The Static: Remixed' (Remix of album already in dataframe)
- 'Jack Johnson And Friends: Sing-A-Longs And Lullabies For The Film Curious George' (album already in dataframe)
<br><br>
10. James Taylor
- No duplicates
<br><br>
11. Khalid
- No duplicates
<br><br>
12. Kid Cudi
- 'The Boy Who Flew To The Moon (Vol. 1)' (Compilation album)
<br><br>
13. Pink Floyd
- No duplicates
<br><br>
14. Post Malone
- No duplicates
<br><br>
15. Simon & Garfunkel
- 'The Graduate' (Mainly consists of songs found in other albums)
<br><br>
16. The Beatles
- No duplicates
<br><br>
17. The Chainsmokers
- No duplicates
<br><br>
18. The Weeknd
- 'Trilogy' (compilation album)
<br><br>
19. blackbear
- No duplicates

In [10]:
# Record initial length of albums_df for assertion statement
initial_len = albums_df.shape[0]

# Compile albums to drop into a list
albums_to_drop = ['I Am...World Tour', 'Above And Beyoncé Dance Mixes', "B'Day", 
                  'Kaya - Deluxe Edition', 'Lights 10', 'Halcyon', 
                  'Bright Lights (Lights Re-pack / Bonus Version)', "Revenge Of The Dreamers III: Director's Cut", 
                  'En Concert', 'Sleep Through The Static: Remixed', 
                  'Jack Johnson And Friends: Sing-A-Longs And Lullabies For The Film Curious George', 
                  'The Boy Who Flew To The Moon (Vol. 1)', 'The Graduate', 'Trilogy']

# Create list of album_ids to drop (for tracks_df)
album_ids_to_drop = albums_df.loc[albums_df.album_name.isin(albums_to_drop)].album_id.values

# Reassign albums_df to dataframe of all albums that are not in the list above
albums_df = albums_df.loc[~albums_df.album_id.isin(album_ids_to_drop)]
assert albums_df.shape[0] == initial_len - len(albums_to_drop)

In [11]:
# Record initial length of tracks_df and number of albums to be dropped for assertion statement
initial_len = tracks_df.shape[0]
num_dropped = tracks_df.loc[tracks_df.album_id.isin(album_ids_to_drop)].shape[0]

# Create list of track_ids to drop (for track_features_df)
track_ids_to_drop = tracks_df.loc[tracks_df.album_id.isin(album_ids_to_drop)].track_id.values

# Drop songs in albums above from tracks_df
tracks_df = tracks_df.loc[~tracks_df.track_id.isin(track_ids_to_drop)]
assert tracks_df.shape[0] == initial_len - num_dropped

In [12]:
# Record initial length of track_features_df and number of tracks to be dropped for assertion statement
initial_len = track_features_df.shape[0]
num_dropped = track_features_df.loc[track_features_df.track_id.isin(track_ids_to_drop)].shape[0]

# Drop songs in albums above from track_features_df
track_features_df = track_features_df.loc[~track_features_df.track_id.isin(track_ids_to_drop)]
assert track_features_df.shape[0] == initial_len - num_dropped

### 2.2: Remove duplicate songs

There are a coule different reasons I don't do a simple DataFrame.duplicated() search here - the first is that this would remove identically named songs even if the artists were different, which are not true duplicates. Additionally, it's possible that just because an artist has multiple songs with the same name, doesn't necessarily mean they are duplicates (looking at the results after the fact, we see that J. Cole has five songs named 'Intro', but just looking at the durations you can tell that they aren't true duplicates.

Lastly, this process helped me discover compilation albums that I did not realize were compilation albums (it's not evident anymore because I went back and removed the albums, but I discovered two: 'Lights 10' by Ellie Goulding and 'The Boy Who Flew To The Moon (Vol. 1)' by Kid Cudi.

In [13]:
# List to contain compiled duplicate track_ids to remove
track_ids_to_drop = []

# Loop through each artist id, and keep same formatting (count, then tab, then artist name) as before
for n, artist_id in enumerate(albums_df['artist_id'].unique()):
    
    # Create list of unique album_ids for each artist
    artist_album_ids = albums_df.loc[albums_df.artist_id == artist_id]['album_id'].values
    
    # Create list of unique song names for each artist
    song_names = []
    for album_id in artist_album_ids:
        song_names.extend(tracks_df.loc[tracks_df.album_id == album_id].song_name.values)
        
    # Create list of duplicate songs (by song_name) for each artist
    duplicate_songs = [song for song, count in collections.Counter(song_names).items() if count > 1]
    
    # Print artist name, dataframe of duplicate songs, and list of track_ids to remove above each table of 
    # duplicate tracks if artist has duplicate songs, otherwise display nothing
    if duplicate_songs:
        print(n, '  ', artists_df.loc[artists_df.artist_id == artist_id]['artist_name'].values[0])  # Artist name
        
        # Duplicate songs dataframe
        dups_df = tracks_df.loc[tracks_df.song_name.isin(duplicate_songs)]
        # Add album_name to dups_df
        dups_df = dups_df.merge(albums_df[['album_id', 'album_name']], on='album_id')
        # Only display relevant rows
        display(dups_df[['song_name', 'album_name', 'duration_ms', 'track_id', 'album_id']])
        
        # Skip dups_df dataframe if song_name == 'Intro' (J. Cole - not true duplicates)
        if 'Intro' in dups_df.song_name.values:
            continue
            
        # Create list of duplicated track_ids (use default 'keep' value of 'first')
        artist_track_ids_to_drop = dups_df[dups_df.duplicated(subset='song_name')]['track_id']
        print(artist_track_ids_to_drop)  # Artist track_ids to remove
        
        # Add artist's duplicated tracks to global compiled duplicated track_ids list
        track_ids_to_drop.extend(artist_track_ids_to_drop)
        print('\n')

5    Doja Cat


,song_name,album_name,duration_ms,track_id,album_id
0,Juicy,Hot Pink,203093,7f5trao56t7sB7f14QDTmp,1MmVkhiwTH0BkNOU3nw5d3
1,Juicy,Amala (Deluxe Version),199586,4cTm3Ev9bUvy4ChJjB1nhl,3wOMqxNHgkga91RBC7BaZU


1    4cTm3Ev9bUvy4ChJjB1nhl
Name: track_id, dtype: object


7    Ellie Goulding


,song_name,album_name,duration_ms,track_id,album_id
0,Lights - Single Version,Halcyon Days,210633,44906V2WC6k59u06A8CO9D,3KVeczHxWg5YFKb0gS62f2
1,Lights - Single Version,Lights,211800,5qftsSFD6Qgndcx13SSqQj,3duZhvcaoqdNveQYXf9dMV


1    5qftsSFD6Qgndcx13SSqQj
Name: track_id, dtype: object


8    J. Cole


...

1    4VywXu6umkIQ2OS0m1I79y
Name: track_id, dtype: object


11    Khalid


,song_name,album_name,duration_ms,track_id,album_id
0,Better,Free Spirit,229400,339Y993TrFyWyO438be0RY,6KT8x5oqZJl9CcnM66hddo
1,Saturday Nights,Free Spirit,211626,1fHRzxbts9A8J1ergDe7Kz,6KT8x5oqZJl9CcnM66hddo
2,Saturday Nights,Suncity,209546,7vGGAJBkHKxKtOnGjQduDV,4UNwL1B7JoymNUiTFJa52B
3,Better,Suncity,229320,2OpBganfGk2GVdSlRdxzaX,4UNwL1B7JoymNUiTFJa52B


2    7vGGAJBkHKxKtOnGjQduDV
3    2OpBganfGk2GVdSlRdxzaX
Name: track_id, dtype: object


16    The Beatles


,song_name,album_name,duration_ms,track_id,album_id
0,Yellow Submarine - Remastered 2009,Yellow Submarine (Remastered),159720,3oEo8Pqm5IAi8wQfCI5BpR,1gKZ5A1ndFqbcrWtW85cCy
1,All You Need Is Love - Remastered 2009,Yellow Submarine (Remastered),231213,2Z8cY4QpZTFUZxCQCpvNe1,1gKZ5A1ndFqbcrWtW85cCy
2,All You Need Is Love - Remastered 2009,Magical Mystery Tour (Remastered),230386,68BTFws92cRztMS1oQ7Ewj,2BtE7qm1qzM80p9vLSiXkj
3,Yellow Submarine - Remastered 2009,Revolver (Remastered),158880,50xwQXPtfNZFKFeZ0XePWc,3PRoXYsngSwjEQWR5PsHWR


2    68BTFws92cRztMS1oQ7Ewj
3    50xwQXPtfNZFKFeZ0XePWc
Name: track_id, dtype: object




In [14]:
# Record initial length of dataframe
initial_len = tracks_df.shape[0]

# Drop duplicate songs found above from tracks_df
tracks_df = tracks_df.loc[~tracks_df.track_id.isin(track_ids_to_drop)]
assert tracks_df.shape[0] == initial_len - len(track_ids_to_drop)

# Drop duplicate songs found above from track_features_df
track_features_df = track_features_df.loc[~track_features_df.track_id.isin(track_ids_to_drop)]
assert track_features_df.shape[0] == initial_len - len(track_ids_to_drop)

<br>

# Step 3: Storage

The cell below creates a 'spotify.db' file in the data folder if it is not already present there, and connects to it to access the data if it is.

I will run and view the SQL queries throughout this notebook using the pandas.read_sql function, due to the consistency and readability of the output table format.

In [15]:
# Establish connection to SQLite server and create spotify.db file
conn = sqlite3.Connection("data/spotify.db")
c = conn.cursor()

## Part 1: `artist` table

In [16]:
# Create artist SQL table from artists_df dataframe
c.execute("""
CREATE TABLE IF NOT EXISTS artist (
    artist_id VARCHAR,
    artist_name VARCHAR,
    external_url VARCHAR,
    genre VARCHAR,
    image_url VARCHAR,
    followers INT,
    popularity INT,
    type VARCHAR,
    artist_uri VARCHAR
    )
""")

In [17]:
# Display artist table schema
q1 = """
PRAGMA table_info('artist')
"""

pd.read_sql(q1, conn).iloc[:, 1:3]

,name,type
0,artist_id,VARCHAR
1,artist_name,VARCHAR
2,external_url,VARCHAR
3,genre,VARCHAR
4,image_url,VARCHAR
5,followers,INT
6,popularity,INT
7,type,VARCHAR
8,artist_uri,VARCHAR


In [18]:
# Convert artists_df to SQL table and add data to artist table
artists_df.to_sql('artist', if_exists='append', index=False, con=conn)

20

<br>

## Part 2: `album` table

In [19]:
# Create album SQL table from albums_df dataframe
c.execute("""
CREATE TABLE IF NOT EXISTS album (
    album_id VARCHAR,
    album_name VARCHAR,
    external_url VARCHAR,
    image_url VARCHAR,
    release_date DATE,
    total_tracks INT,
    type VARCHAR,
    album_uri VARCHAR,
    artist_id VARCHAR
    )
""")

In [20]:
# Display album table schema
q2 = """
PRAGMA table_info('album')
"""

pd.read_sql(q2, conn).iloc[:, 1:3]

,name,type
0,album_id,VARCHAR
1,album_name,VARCHAR
2,external_url,VARCHAR
3,image_url,VARCHAR
4,release_date,DATE
5,total_tracks,INT
6,type,VARCHAR
7,album_uri,VARCHAR
8,artist_id,VARCHAR


In [21]:
# Convert albums_df to SQL table and add data to album table
albums_df.to_sql('album', if_exists='append', index=False, con=conn)

136

<br>

## Part 3: `track` table

In [22]:
# Create track SQL table from tracks_df dataframe
c.execute("""
CREATE TABLE IF NOT EXISTS track (
    track_id VARCHAR,
    song_name VARCHAR,
    external_url VARCHAR,
    duration_ms INT,
    explicit BOOLEAN,
    disc_number INT,
    type VARCHAR,
    song_uri VARCHAR,
    album_id VARCHAR
    )
""")

In [23]:
# Display track table schema
q3 = """
PRAGMA table_info('track')
"""

pd.read_sql(q3, conn).iloc[:, 1:3]

,name,type
0,track_id,VARCHAR
1,song_name,VARCHAR
2,external_url,VARCHAR
3,duration_ms,INT
4,explicit,BOOLEAN
5,disc_number,INT
6,type,VARCHAR
7,song_uri,VARCHAR
8,album_id,VARCHAR


In [24]:
# Convert tracks_df to SQL table and add data to track table
tracks_df.to_sql('track', if_exists='append', index=False, con=conn)

1890

<br>

## Part 4: `track_feature` table

In [25]:
# Create track_feature SQL table from track_features_df dataframe
c.execute("""
CREATE TABLE IF NOT EXISTS track_feature (
    track_id VARCHAR,
    danceability DOUBLE,
    energy DOUBLE,
    instrumentalness DOUBLE,
    liveness DOUBLE,
    loudness DOUBLE,
    speechiness DOUBLE,
    tempo DOUBLE,
    type VARCHAR,
    valence DOUBLE,
    song_uri VARCHAR
    )
""")

In [26]:
# Display track_feature table schema
q4 = """
PRAGMA table_info('track_feature')
"""

pd.read_sql(q4, conn).iloc[:, 1:3]

,name,type
0,track_id,VARCHAR
1,danceability,DOUBLE
2,energy,DOUBLE
3,instrumentalness,DOUBLE
4,liveness,DOUBLE
5,loudness,DOUBLE
6,speechiness,DOUBLE
7,tempo,DOUBLE
8,type,VARCHAR
9,valence,DOUBLE


In [27]:
# Convert track_features_df to SQL table and add data to track_feature table
track_features_df.to_sql('track_feature', if_exists='append', index=False, con=conn)

1890

In [28]:
# Display all tables and views
q5 = """
SELECT * FROM sqlite_master;
"""

pd.read_sql(q5, conn)

,type,name,tbl_name,rootpage,sql
0,table,artist,artist,2,"CREATE TABLE artist (\n artist_id VARCHAR,\..."
1,table,album,album,5,"CREATE TABLE album (\n album_id VARCHAR,\n ..."
2,table,track,track,15,"CREATE TABLE track (\n track_id VARCHAR,\n ..."
3,table,track_feature,track_feature,100,CREATE TABLE track_feature (\n track_id VAR...


In [29]:
# Commit additions to database
conn.commit()

<br>

### Save dataframes as `pkl` files

In [30]:
if not os.path.exists('data/artists_df.pkl'):
    artists_df.to_pickle('data/artists_df.pkl')
    
if not os.path.exists('data/albums_df.pkl'):
    albums_df.to_pickle('data/albums_df.pkl')
    
if not os.path.exists('data/tracks_df.pkl'):
    tracks_df.to_pickle('data/tracks_df.pkl')
    
if not os.path.exists('data/track_features_df.pkl'):
    track_features_df.to_pickle('data/track_features_df.pkl')